In [1]:
import os, sys
sys.path.insert(0, '../mhdb')

from pycalphad import Database, equilibrium, variables as v
from pymongo import MongoClient
from core import database, mongo
import matplotlib.pyplot as plt

client_string = os.getenv('MONGODB_URI')
client = MongoClient(client_string)

db = client['MHDB'] 
collections = [db['curated'], db['community']]

In [10]:
entries = collections[1].find({})

references = []
for entry in entries:
    reference = entry['metadata']['parentDatabase']
    references.append(reference)

print(f'The MHDB-community currently has {len(set(references))} contributions.', '\n')

for i, reference in enumerate(set(references)):
    entries = collections[1].find({'metadata.parentDatabase': reference})

    systems = []
    for entry in entries:
        system = entry['material']['system']
        if len(system) != 1:
            systems.append(entry['material']['phaseLabel'])

    print(f"[{i+1}] The '{reference.split(' | ')[0]}' contains {len(set(systems))} phases:")
    print(set(systems), '\n')

The MHDB-community currently has 1 contributions. 

[1] The 'La-Ni-H_Ya.tdb' contains 15 phases:
{'LANI', 'BCC', 'LA3NI', 'LA7NI3', 'LANI3', 'LIQUID', 'LANI5H7', 'LA5NI19', 'LA2NI7', 'DHCP', 'FCC', 'LA2NI3', 'LAHX', 'LA7NI16', 'LANI5H1-X'} 



In [5]:
                                                         # INPUT(1): TDB reference and Chemical composition of metal hydride
tdb_reference = 'La-Ni-H_Ya.tdb'

elements = ['La', 'H', 'Va']

In [6]:
regex_pattern = '^(' + '|'.join(elements) + ')(-(' + '|'.join(elements) + '))*$'

data_collection = []
for collection in collections:
    data = collection.find({'$and': [{'material.system': {'$regex': regex_pattern, '$options': 'i'}}, {'metadata.parentDatabase': {'$regex': tdb_reference, '$options': 'i'}}]}, {'tdb': 1, '_id': 0})
    data_collection.extend(entry['tdb'] for entry in data)

try:
    data = database.many2one(elements, data_collection)
    data = database.one2tdb(data)

    dbf = Database(data)
    phases = list(dbf.phases.keys())
    print(f'Data for the following phases were successfully retrieved: {phases}')

except:
    print('No dataset found')

Data for the following phases were successfully retrieved: ['GAS', 'LIQUID', 'BCC', 'DHCP', 'FCC', 'LAHX']


In [7]:
print(dbf)

Elements: ['/-', 'H', 'LA', 'VA']
Species: [Species('/-', '/-1'), Species('H', 'H1'), Species('H2', 'H2.0'), Species('LA', 'LA1'), Species('VA', 'VA1')]
Phase({'name': 'BCC', 'constituents': (frozenset({Species('LA', 'LA1')}), frozenset({Species('VA', 'VA1'), Species('H', 'H1')})), 'sublattices': (1.0, 3.0), 'model_hints': {}})
Phase({'name': 'DHCP', 'constituents': (frozenset({Species('LA', 'LA1')}),), 'sublattices': (1.0,), 'model_hints': {}})
Phase({'name': 'FCC', 'constituents': (frozenset({Species('LA', 'LA1')}), frozenset({Species('VA', 'VA1'), Species('H', 'H1')})), 'sublattices': (1.0, 1.0), 'model_hints': {}})
Phase({'name': 'GAS', 'constituents': (frozenset({Species('H2', 'H2.0')}),), 'sublattices': (1.0,), 'model_hints': {'gas': True}})
Phase({'name': 'LAHX', 'constituents': (frozenset({Species('LA', 'LA1')}), frozenset({Species('VA', 'VA1'), Species('H', 'H1')}), frozenset({Species('VA', 'VA1'), Species('H', 'H1')})), 'sublattices': (0.25, 0.5, 0.25), 'model_hints': {}})
Ph